In [ ]:
!pip install streamlit
!pip install pyngrok

In [2]:
%%writefile app.py

import streamlit as st
import requests
import time
import random
from bs4 import BeautifulSoup
import pandas as pd
from random import randint
from time import sleep
import base64

def get_soup(url):
    USER_AGENTS = [
      ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'),
      ('Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.110 Safari/537.36'),  # chrome
      ('Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.79 Safari/537.36'),  # chrome
      ('Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:55.0) Gecko/20100101 Firefox/55.0')  # firefox
    ]
    ind = random.randint(0, len(USER_AGENTS) - 1)
    headers = {'User-Agent': USER_AGENTS[ind]}

    try:
        response = requests.get(url, headers=headers, timeout = 10)
        soup = BeautifulSoup(response.text)
    except:
        print("Impossible de se connecter", response.status_code)
    return soup


def scrape_offers(offer_title, offer_location, offer_type, offer_date):
    more_links = True
    page = 1
    offers = []
    offer_url = f'https://api.scraperapi.com?api_key={tokens["scraperapi"]}&url=https://www.indeed.fr/voir-emploi?jk='
    #if offer_title == 'all':
    #  url = f'https://api.scraperapi.com?api_key={tokens["scraperapi"]}&url=https://fr.indeed.com/emplois?q={offer_title}&l={offer_location}&fromage={offer_date}&render=true'
    #else:
    api_flags = "&render=true&keep_headers=true"
    url = f'https://api.scraperapi.com?api_key={tokens["scraperapi"]}&url=https://fr.indeed.com/emplois?q={offer_title}&l={offer_location}&jt={offer_type}&fromage={offer_date}{api_flags}'

    USER_AGENTS = [
      ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'),
      ('Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.110 Safari/537.36'),  # chrome
      ('Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.79 Safari/537.36'),  # chrome
      ('Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:55.0) Gecko/20100101 Firefox/55.0')  # firefox
    ]
    ind = random.randint(0, len(USER_AGENTS) - 1)
    headers = {'User-Agent': USER_AGENTS[ind]}

    while(more_links):
        response = requests.get(url, headers=headers, timeout = 100)
        soup = BeautifulSoup(response.text)

        for item in soup.select('a.tapItem'):
            offer = {}

            #On récupère le dernier element pour éviter de récupérer le tag "nouveau"
            offer['title'] = item.select_one('h2.jobTitle').find_all("span")[-1].text.strip()

            offer['link'] = item['data-jk']

            try:
              offer['date'] = item.select_one('span.date').text.strip()
            except: 
              offer['date'] = 'None'

            try:
              offer['company'] = item.select_one('span.companyName').text.strip()
            except: 
              offer['company'] = 'None'

            try:
              offer['location'] = item.select_one('div.companyLocation').text
            except: 
              offer['location'] = 'None'

            try:
              offer['summary'] = item.select_one('div.job-snippet').text.strip()
            except: 
              offer['summary'] = 'None'
            
            try:
              offer['rate'] = item.select_one('span.ratingNumber').text.strip()
            except: 
              offer['rate'] = 'None'

            #detailedResponse = requests.get(offer_url+offer['link'], headers=headers, timeout = 100)
            #detailedSoup = BeautifulSoup(detailedResponse.text, 'html.parser')
            #
            #offer['description'] = []
            #for tag in detailedSoup.select('div.jobsearch-jobDescriptionText'):
            #  try:
            #      offer['description'].append([''.join(tag.text.strip())])
            #  except AttributeError:
            #      offer['description'].append([''])

            offers.append(offer)
            sleep(randint(1,8))

        next_link = soup.findAll("a", {"aria-label" : "Suivant"})
        if(next_link):
            page += 1
            url = f"https://api.scraperapi.com?api_key={tokens['scraperapi']}&url=https://fr.indeed.com/"+next_link[0].get('href')+api_flags
        else:
            more_links = False

    print(f'{page} pages scrappé(s)')
    return offers

def main():
  st.set_page_config(page_title='Indeed Scrapper')
  st.title("Indeed Scraper")
  st.subheader("Faites votre recherche")
  offer_title_st = st.sidebar.text_input("Poste")
  offer_types = ['Temps plein','CDI','CDD','Apprentissage','Contrat pro', 'Intérim', 'Freelance / Indépendant']
  offer_type_st = st.sidebar.selectbox('Type de contrat',offer_types)

  #if offer_type_st == "Tous":
  #  offer_type = "all"
  if offer_type_st == "Temps plein":
    offer_type = "fulltime"
  elif offer_type_st == "CDI":
    offer_type = "permanent"
  elif offer_type_st == "CDD":
    offer_type = "contract"
  elif offer_type_st == "Apprentissage":
    offer_type = "apprenticeship" 
  elif offer_type_st == "Contrat pro":
    offer_type = "custom_1"
  elif offer_type_st == "Intérim":
    offer_type = "temporary"
  elif offer_type_st == "Freelance / Indépendant":
    offer_type = "subcontract"
  
  offer_location_st = st.sidebar.text_input("Localisation")
  offer_dates = ['Tout','Denières 24 heures','3 derniers jours','7 derniers jours','14 derniers jours']
  offer_date_st = st.sidebar.selectbox("Date de publication",offer_dates)

  if offer_date_st == 'Tout':
      offer_date = "last"
  elif offer_date_st == 'Denières 24 heures':
      offer_date = 1
  elif offer_date_st == '3 derniers jours':
      offer_date = 3
  elif offer_date_st == '7 derniers jours':
      offer_date = 7
  elif offer_date_st == '14 derniers jours':
      offer_date = 14

  search_st = st.sidebar.button("Rechercher")
  if search_st:
    offer_title = offer_title_st.replace(" ","+")
    offer_location = offer_location_st.replace(" ", "+")
    #st.markdown(f'https://api.scraperapi.com?api_key={tokens["scraperapi"]}&url=https://fr.indeed.com/emplois?q={offer_title}&l={offer_location}&jt={offer_type}&fromage={offer_date}&render=true')
    with st.spinner('En cours de récupération...'):
      offers = scrape_offers(offer_title, offer_location, offer_type, offer_date)
      df = pd.DataFrame(offers)
      list_of_col_names = ['title', 'company', 'location', 'summary', 'date', 'rate', 'description', 'link']
      df = df.filter(list_of_col_names)
      df.to_csv('offers.csv')

      csv = df.to_csv(index=False)
      b64 = base64.b64encode(csv.encode()).decode()
      href = f'<a href="data:file/csv;base64,{b64}" download="offers.csv">Télécharger au format csv</a>'

    st.success('Terminé !')
    st.markdown(href, unsafe_allow_html=True)
    st.dataframe(df)
  
if __name__ == '__main__':
  tokens = {}
  with open('pwd.txt') as f:
    for line in f:
        x = line.split(":")
        tokens[x[0]] = x[1].rstrip("\n")

  main()

Overwriting app.py


In [3]:
!ngrok authtoken tokens['ngrok']

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
#!streamlit run app.py &>/dev/null&
!streamlit run --server.port 80 app.py &>/dev/null& 

In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect(port='8501')

In [ ]:
public_url

<NgrokTunnel: "http://7f4dc5db6379.ngrok.io" -> "http://localhost:80">

In [ ]:
!pgrep streamlit

154


In [ ]:
!kill 154
ngrok.kill()